In [ ]:
import numpy as np 
import pandas as pd 
from PIL import Image
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.preprocessing import minmax_scale
import random
import cv2
import warnings
warnings.filterwarnings('ignore')

import albumentations as A
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.applications import EfficientNetB4,EfficientNetB7
from tensorflow.keras.layers.experimental.preprocessing import RandomCrop,CenterCrop, RandomRotation
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.losses import CategoricalCrossentropy


In [ ]:
from sklearn import preprocessing
train_df=pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
train_image_path = '../input/plant-pathology-2021-fgvc8/train_images/'
train_df['filepath'] = train_image_path+train_df['image']



train_df

In [ ]:
Initial = train_df['labels'].values.copy()

In [ ]:
from collections import defaultdict


dct = defaultdict(list)

for i, labels in enumerate(train_df.labels):
    for category in labels.split():
        dct[category].append(i)
 
dct = {key: np.array(val) for key, val in dct.items()}
dct

In [ ]:
new_df = pd.DataFrame(np.zeros((train_df.shape[0], len(dct.keys())), dtype=np.int8), columns=dct.keys())

for key, val in dct.items():
    new_df.loc[val, key] = 1
    
train_df = pd.concat([train_df, new_df], axis=1)
train_df=train_df.drop(['labels'],axis=1)
train_df=train_df.drop(['image'],axis=1)

train_df

In [ ]:
batch_size = 8 # 배치 사이즈를 설정
image_size = 512 # 이미지의 크기를 설정
input_shape = (image_size, image_size, 3) #이미지의 사이즈 정의 (컬러 이미지이기 때문에 한 화소당 3개의 데이터가 필요)
dropout_rate = 0.35 #드롭아웃 비율 정의
classes_to_predict = sorted("healthy scab frog_eye_leaf_spot complex rust powdery_mildew".split(), key=str.lower) #예측해야 하는 클래스 수 정의, 여기서는 12개
classes_to_predict

In [ ]:
final_train_index = list() #어떤 train 데이터를 고를지 저장하는 list를 정의
final_test_index = list() #어떤 test 데이터를 고를지 저장하는 list를 정의for i, (train_index, val_index) in enumerate(kfold.split(df.index, original_labels)):
fold=np.zeros((len(train_df),))
skf = StratifiedKFold(n_splits=4) #4 fold를 하는 stratifiedKFold를 선언 (validation 25%)
for i, (train_index, test_index) in enumerate(skf.split(train_df.index,Initial)): #각 fold에 해당하는 데이터를 출력
    fold[test_index]=i
    final_train_index.append(train_index) #final_train_index에 각 fold에 해당하는 데이터를 입력
    final_test_index.append(test_index) #final_test_index에 각 fold에 해당하는 데이터를 입력

In [ ]:
training_df = train_df.iloc[final_train_index[0]] #train 데이터를 만듬
validation_df = train_df.iloc[final_test_index[0]] #validation 만듬

In [ ]:
training_data = tf.data.Dataset.from_tensor_slices((training_df.filepath.values, training_df.loc[:,train_df.columns!="filepath"].values))
validation_data = tf.data.Dataset.from_tensor_slices((validation_df.filepath.values, validation_df.loc[:,train_df.columns!="filepath"].values))
print(training_data)



In [ ]:
"""def load_image_and_label_from_path(image_path, label): #이미지 데이터를 불러와 텐서 (array와 비슷한 형태)로 변환하는 함수
    img = tf.io.read_file(image_path) #이미지 경로의 파일을 읽음
    img = tf.image.decode_jpeg(img, channels=3) #이미지를 array 데이터로 변환하여 저장
    img = tf.image.random_crop(img, size=[image_size,image_size,3]) # 이미지를 랜덤으로 원하는 사이즈로 잘라줌. 중앙만 자르고 싶다면 central_crop 사용.
    return img, label

AUTOTUNE = tf.data.experimental.AUTOTUNE #메모리 동적 할당을 위한 AUTOTUNE
training_data = training_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE) #train 데이터를 불러옴
validation_data = validation_data.map(load_image_and_label_from_path,num_parallel_calls=AUTOTUNE) #validation 데이터를 불러옴 """

In [ ]:
def load_image_and_label_from_path(image_path, label): #이미지 데이터를 불러와 텐서 (array와 비슷한 형태)로 변환하는 함수
    src = cv2.imread(image_path, cv2.IMREAD_COLOR)
    dst = cv2.cvtColor(src, cv2.COLOR_BGR2HSV) #이미지 경로의 파일을 읽음
    img = tf.image.decode_jpeg(dst, channels=3) #이미지를 array 데이터로 변환하여 저장
    img = tf.image.random_crop(img, size=[image_size,image_size,3]) # 이미지를 랜덤으로 원하는 사이즈로 잘라줌. 중앙만 자르고 싶다면 central_crop 사용.
    return img, label

AUTOTUNE = tf.data.experimental.AUTOTUNE #메모리 동적 할당을 위한 AUTOTUNE
training_data = training_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE) #train 데이터를 불러옴
validation_data = validation_data.map(load_image_and_label_from_path,num_parallel_calls=AUTOTUNE) #validation 데이터를 불러옴

src = cv2.imread("Image/crow.jpg", cv2.IMREAD_COLOR)
dst = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

In [ ]:
#train 및 validation 데이터를 훈련하기 좋게 batch로 자름
training_data_batches = training_data.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=AUTOTUNE)
validation_data_batches = validation_data.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [ ]:
import imgaug.augmenters as iaa

In [ ]:
augmenter = iaa.Sequential([
    iaa.color.AddToHueAndSaturation(value=(-20, 20)),
    iaa.KMeansColorQuantization(n_colors=16),
    
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Cutout(nb_iterations=(0,2), size=0.2, fill_mode="gaussian", fill_per_channel=True),
    iaa.GaussianBlur(sigma=(0,0.3)),
    iaa.MultiplyBrightness((0.7,1.2)),
    iaa.PiecewiseAffine(scale=(0,0.06)),
    iaa.AddElementwise((-20, 20), per_channel=0.3),
    iaa.Add((-20, 20), per_channel=0.2),
    iaa.ReplaceElementwise(0.1, [40, 255],per_channel=0.2),
  ])

In [ ]:
def augment_fn():
    def augment(images, labels):
        img_dtype = images.dtype
        img_shape = tf.shape(images)
        images = tf.numpy_function(augmenter.augment_images,
                                   [images],
                                   img_dtype)
        images = tf.reshape(images, shape = img_shape)
        return images, labels
    return augment

In [ ]:
training_data_batches = training_data_batches.map(augment_fn())
training_data_batches

In [ ]:
data_augmentation_layers = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"), #랜덤으로 이미지를 좌우로 뒤집어줌.
        layers.experimental.preprocessing.RandomRotation(0.3), #이미지를 좌우로 25% 이내로 랜덤으로 돌립니다. 
        layers.experimental.preprocessing.RandomZoom((-0.2, 0.1)), #이미지를 0~20%만큼 랜덤으로 축소합니다.
        layers.experimental.preprocessing.RandomContrast((0.2,0.2))
    ]
)

In [ ]:
efficientnet = EfficientNetB4(weights="imagenet", #이미지넷 가중치 값을 불러와 적용
                              include_top=False, 
                              input_shape=input_shape, 
                              drop_connect_rate=dropout_rate) #efficientnetB4 모델을 로드
efficientnet.trainable=False# efficientnetB4의 학습을 허용. 만약 False로 지정할 시에 정확도는 떨어지지만 학습 속도가 매우 빨라짐.

In [ ]:
model = Sequential() #새 Sequential 모델을 만듬 
model.add(Input(shape=input_shape)) #인풋을 이미지 사이즈로 설정
model.add(data_augmentation_layers) #이미지 augumentation 레이어 추가
model.add(efficientnet) # efficientnetb0 추가
model.add(layers.GlobalAveragePooling2D()) # 풀링 레이어를 추가
model.add(layers.Dropout(dropout_rate)) # 드롭아웃 레이어를 추가
model.add(Dense(len(classes_to_predict), activation="sigmoid")) #마지막 덴스 레이어를 추가. 예측할 클래스의 개수만큼이 아웃풋이 된다. 
model.summary() #모델 확인

In [ ]:
import tensorflow.keras.metrics as _metrics

In [ ]:
loss = CategoricalCrossentropy(label_smoothing=0.08)

In [ ]:
epochs = 15 #에폭 수를 설정합니다.
decay_steps = int(round(len(training_df)/batch_size))*epochs
cosine_decay = CosineDecay(initial_learning_rate=1e-4, decay_steps=decay_steps, alpha=0.3) #learning rate를 에폭이 지날수록 점점 줄여나가는 cosine decay 방법을 사용합니다. 
callbacks = [ModelCheckpoint(filepath='mymodel.h5', monitor='Mean F1-Score', save_best_only=True), #가장 validation loss가 낮은 에폭의 모델을 .h5 파일로 저장합니다. 
            EarlyStopping(monitor='Mean F1-Score', patience = 5, verbose=2)] #정해진 에폭이 되기 전에 5번의 에폭동한 validation loss가 향상되지 않으면 학습을 종료합니다. 

model.compile(loss=loss, optimizer=Adam(cosine_decay), metrics=["accuracy",tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]) #loss는 sparse_categorical_crossentropy, optimizer는 Adam을 사용합니다. 각 에폭당 정확도를 통해 모델의 성능을 모니터링합니다,

In [ ]:
history = model.fit(training_data_batches, #모델을 학습합니다. 
                  epochs = epochs, 
                  validation_data=validation_data_batches,
                  callbacks=callbacks)

In [ ]:
model.save('B4_onehotencoding.h5')